<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Tutorials/Customization/tf.function/TensorFlow_2_0_Training_with_tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 380.8MB 40kB/s 
     |████████████████████████████████| 3.8MB 34.2MB/s 
     |████████████████████████████████| 450kB 66.4MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exist

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model

(X_train, y_train),(X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
X_train, X_test = X_train[..., tf.newaxis], X_test[..., tf.newaxis]
X_train, X_test = X_train/255.0, X_test/255.0

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache().batch(128).shuffle(10000).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).cache().batch(128).prefetch(tf.data.experimental.AUTOTUNE)

### without decorating call method

In [0]:
%%time
class MyModel(tf.keras.Model):
  def __init__(self, *args, **kwargs):
    super(MyModel, self).__init__(args, kwargs)
    self.conv1 = Conv2D(256,(3,3),activation=tf.nn.relu)
    self.conv2 = Conv2D(128,(3,3),activation=tf.nn.relu)
    self.maxpool1 = MaxPooling2D(2,2)
    self.maxpool2 = MaxPooling2D(2,2)
    self.flatten = Flatten()
    self.dense1 = Dense(1024,activation=tf.nn.relu)
    self.dense2 = Dense(512,activation=tf.nn.relu)
    self.out = Dense(10,activation=tf.nn.softmax)

  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    return self.out(x)

model = MyModel()
model.build((512,28,28,1))
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

EPOCHS = 10

for epoch in tf.range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  model.reset_metrics()
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  
  template = 'Epoch {}, Train Accuracy: {}, Test Accuracy: {}, Train Loss {}, Test Loss{}'
  print(template.format(epoch+1, 
                        train_accuracy.result(), 
                        test_accuracy.result(), 
                        train_loss.result(), 
                        test_loss.result()))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Train Accuracy: 0.9564333558082581, Test Accuracy: 0.9810000061988831, Train Loss 0.13982629776000977, Test Loss0.057740554213523865
Epoch 2, Train Accuracy: 0.9722583293914795, Test Accuracy: 0.9840499758720398, Train Loss 0.08932305127382278, Test Loss0.0505184568464756
Epoch 3, Train Accuracy: 0.978755533695221, Test Accuracy: 0.9854666590690613, Train Loss 0.06865186989307404, Test Loss0.04607753828167915
Epoch 4, Train Accuracy: 0.9826291799545288, Test Accuracy: 0.9865249991416931, Train 

### decorating call method with tf.function

In [0]:
%%time
class MyModel(tf.keras.Model):
  def __init__(self, *args, **kwargs):
    super(MyModel, self).__init__(args, kwargs)
    self.conv1 = Conv2D(256,(3,3),activation=tf.nn.relu)
    self.conv2 = Conv2D(128,(3,3),activation=tf.nn.relu)
    self.maxpool1 = MaxPooling2D(2,2)
    self.maxpool2 = MaxPooling2D(2,2)
    self.flatten = Flatten()
    self.dense1 = Dense(1024,activation=tf.nn.relu)
    self.dense2 = Dense(512,activation=tf.nn.relu)
    self.out = Dense(10,activation=tf.nn.softmax)

  @tf.function
  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    return self.out(x)

model = MyModel()
model.build((512,28,28,1))

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

EPOCHS = 10

for epoch in tf.range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  model.reset_metrics()
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  
  template = 'Epoch {}, Train Accuracy: {}, Test Accuracy: {}, Train Loss {}, Test Loss{}'
  print(template.format(epoch+1, 
                        train_accuracy.result(), 
                        test_accuracy.result(), 
                        train_loss.result(), 
                        test_loss.result()))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Train Accuracy: 0.9565500020980835, Test Accuracy: 0.9850000143051147, Train Loss 0.13509458303451538, Test Loss0.04842051491141319
Epoch 2, Train Accuracy: 0.9721083045005798, Test Accuracy: 0.9867500066757202, Train Loss 0.08745210617780685, Test Loss0.04094132035970688
Epoch 3, Train Accuracy: 0.9787333607673645, Test Accuracy: 0.9880333542823792, Train Loss 0.06680717319250107, Test Loss0.035799168050289154
Epoch 4, Train Accuracy: 0.98253333568573, Test Accuracy: 0.9890750050544739, Train 